# Queries de SQL aplicadas sobre la BBDD

El siguiente notebook busca responder las consignas del punto 2. Se ejecutarán las queries pertinentes, y se mostrarán en pantalla los resultados. 

*Observación: para correr adecuadamente este notebook, se requieren las credenciales de acceso a la base, las cuales no están publicadas en el repositorio público. A su vez, se deben instalar las bibliotecas que se importan a continuación*


In [6]:
# Importamos bibliotecas...
import pandas as pd
import sqlalchemy as sql
import os
from dotenv import load_dotenv

In [8]:
# Cargamos las credenciales de la base de datos
load_dotenv()
load_dotenv()

try:
    # Variables de entorno
    NAME = os.getenv("NAME")
    PASSWORD = os.getenv("PASSWORD")
    USER = os.getenv("USERNAME_AIVEN")
    HOST = os.getenv("HOST")
    PORT = os.getenv("PORT") 
    SSLMODE = os.getenv("SSL_MODE")  

    # Verificación de que todas las variables fueron cargadas correctamente
    if not all([NAME, PASSWORD, USER, HOST, PORT, SSLMODE]):
        raise ValueError("Algunas de las variables de entorno no están correctamente definidas.")

    # Creamos la conexión
    engine = sql.create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{NAME}?sslmode={SSLMODE}")
    print("Base cargada con exito")


except ValueError as ve:
    print(f"Error en las variables de entorno: {ve}")

except sql.exc.SQLAlchemyError as sae:
    print(f"Error al conectarse a la base de datos: {sae}")

except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")


Base cargada con exito


In [3]:
# Query de prueba
query = "SELECT * FROM pacientes LIMIT 5"
pd.read_sql(query, engine, index_col="id_paciente") 

,nombre,fecha_nacimiento,id_sexo,numero,calle,ciudad,edad
id_paciente,,,,,,,
1,Luciana Gómez,1991-07-12,2,500,Calle Corrientes,Buenos Aires,33
5,Julieta Rodríguez,1985-06-18,2,845,Calle Mitre,Buenos Aires,39
6,Santiago Pérez,1979-02-09,1,1103,Calle Balcarce,Buenos Aires,45
13,Micaela Gutiérrez,1988-05-14,2,776,Avenida Sarmiento,Buenos Aires,36
14,Nicolás Morales,1992-10-11,1,923,Calle Rivadavia,Buenos Aires,32


**1. cuando se realizan consultas sobre la tabla paciente agrupando por ciudad los tiempos de respuesta son demasiado largos. 
proponer mediante una query sql una solución a este problema.**

In [16]:
query_1 = '''CREATE INDEX IF NOT EXISTS idx_ciudades 
             ON pacientes (ciudad);'''

with engine.connect() as connection:
    try:
        connection.execute(sql.text(query_1))
        print("Índice creado correctamente")
        
    except Exception as e:
        print(e)

Índice creado correctamente


In [18]:
query_2_1 = '''ALTER TABLE pacientes
                ADD COLUMN IF NOT EXISTS edad INT;'''
query_2_2 = '''UPDATE pacientes
                SET edad = EXTRACT(YEAR FROM AGE(CURRENT_DATE, fecha_nacimiento));'''
query_2_3 = '''CREATE INDEX IF NOT EXISTS idx_edades
                ON pacientes (edad);''' 

with engine.connect() as connection:
    try:
        connection.execute(sql.text(query_2_1))
        connection.execute(sql.text(query_2_2))
        connection.execute(sql.text(query_2_3))
        print("Columna e Índice creados correctamente")
        
    except Exception as e:
        print(e)

Columna e Índice creados correctamente


**3. La paciente, “Luciana Gómez”, ha cambiado de dirección. Antes vivía en “Avenida Las Heras 121” en “Buenos Aires”, pero ahora vive en “Calle Corrientes 500” en “Buenos Aires”. Actualizar la dirección de este paciente en la base de datos.**

In [21]:
query_3 =  '''UPDATE pacientes
                SET calle = 'Calle Corrientes', numero = 500
                WHERE nombre = 'Luciana Gómez';'''

with engine.connect() as connection:
    try:
        res = connection.execute(sql.text(query_3))
        print("Datos actualizados correctamente")
        print(f"{res.rowcount} fila(s) afectadas") 

    except Exception as e:
        print(e)

Datos actualizados correctamente
1 fila(s) afectadas


**4. Seleccionar el nombre y la matrícula de cada médico cuya especialidad sea identificada por el id 4.**

In [22]:
query_4 = '''SELECT m.nombre, m.matricula, e.nombre
            FROM medicos m JOIN especialidades e
            ON m.especialidad_id = e.id_especialidad
            WHERE e.id_especialidad = 4;'''
pd.read_sql(query_4, engine)

,nombre,matricula,nombre
0,Dra. Lucía Rodríguez,89012,Dermatología
1,Dr. Nicolás Gutiérrez,90123,Dermatología


**5. Puede pasar que haya inconsistencias en la forma en la que están escritos los nombres de las ciudades, ¿cómo se corrige esto? Agregar la query correspondiente.**

Para esta actividad, haremos uso de la biblioteca `pg_trgm`, la cual debemos importar a la base:

In [27]:
query_5_1 = "CREATE EXTENSION IF NOT EXISTS pg_trgm;"
query_5_2 = "SELECT * FROM pg_extension WHERE extname = 'pg_trgm';"

with engine.connect() as connection:
    try:
        connection.execute(sql.text(query_5_1))
        res = connection.execute(sql.text(query_5_2))
        print("Extensión creada correctamente")
        print(res.fetchall())
        connection.execute(sql.text('COMMIT;'))
        
    except Exception as e:
        print(e)

Extensión creada correctamente
[(16615, 'pg_trgm', 10, 2200, True, '1.6', None, None)]


In [67]:
ciudades = ['Buenos Aires', 'Rosario', 'Córdoba', 'Sante Fé', 'Mendoza']

with engine.connect() as connection:
    for ciudad in ciudades:
        query_5 = f'''UPDATE pacientes
                    SET ciudad = '{ciudad}'
                    WHERE similarity(ciudad, '{ciudad}') > 0.3;'''
        try:
            res = connection.execute(sql.text(query_5))
            print(f"Datos actualizados correctamente")
            print(f"{res.rowcount} fila(s) afectadas")
        except Exception as e:
            print(f"Error en la ciudad {ciudad}")
            print(e)


Datos actualizados correctamente
8 fila(s) afectadas
Datos actualizados correctamente
4 fila(s) afectadas
Datos actualizados correctamente
6 fila(s) afectadas
Datos actualizados correctamente
1 fila(s) afectadas
Datos actualizados correctamente
4 fila(s) afectadas


**6. Obtener el nombre y la dirección de los pacientes que viven en Buenos Aires.**

In [37]:
query_6 = '''SELECT nombre, calle || ' ' || numero AS direccion
            FROM pacientes
            WHERE ciudad = 'Buenos Aires';'''
pd.read_sql(query_6, engine)

,nombre,direccion
0,Luciana Gómez,Calle Corrientes 500
1,Julieta Rodríguez,Calle Mitre 845
2,Santiago Pérez,Calle Balcarce 1103
3,Micaela Gutiérrez,Avenida Sarmiento 776
4,Nicolás Morales,Calle Rivadavia 923
5,Carolina Figueroa,Calle Rivadavia 135
6,Agustín Romero,Calle 25 de Mayo 853
7,Sofía Maldonado,Avenida Libertador 492


**7. Cantidad de pacientes que viven en cada ciudad.**

In [38]:
query_7 = ''' SELECT ciudad, COUNT(*) AS cantidad
            FROM pacientes
            GROUP BY ciudad
            ORDER BY cantidad DESC;'''
pd.read_sql(query_7, engine)

,ciudad,cantidad
0,Buenos Aires,8
1,Córdoba,6
2,Mendoza,4
3,Rosario,4
4,Sante Fé,1


**8. Cantidad de pacientes por sexo que viven en cada ciudad.**

In [50]:
query_8 = '''SELECT ciudad, descripcion, COUNT(*) AS cantidad
            FROM pacientes JOIN sexobiologico
            ON pacientes.id_sexo = sexobiologico.id_sexo
            GROUP BY ciudad, descripcion;'''
pd.read_sql(query_8, engine)

,ciudad,descripcion,cantidad
0,Buenos Aires,Masculino,3
1,Mendoza,Masculino,2
2,Córdoba,Femenino,3
3,Rosario,Masculino,3
4,Rosario,Femenino,1
5,Mendoza,Femenino,2
6,Sante Fé,Masculino,1
7,Buenos Aires,Femenino,5
8,Córdoba,Masculino,3


**9. Obtener la cantidad de recetas emitidas por cada médico.**

In [54]:
query_9 = '''SELECT m.id_medico, nombre, COUNT(id_receta) AS cantidad
            FROM medicos m LEFT JOIN recetas
            ON m.id_medico = recetas.id_medico
            GROUP BY m.id_medico, nombre
            ORDER BY cantidad DESC;'''
pd.read_sql(query_9, engine, index_col="id_medico")

,nombre,cantidad
id_medico,,
9,Dr. Nicolás Gutiérrez,6
8,Dra. Lucía Rodríguez,5
2,Dra. Laura Fernández,4
10,Dra. Mónica Silva,3
7,Dra. Carolina Méndez,3
1,Dr. Carlos García,3
3,Dr. Pedro Ruiz,3
4,Dra. Gabriela Fernández,3
12,Dra. Valentina López,0


**10. Obtener todas las consultas médicas realizadas por el médico con ID igual a 3 durante el mes de agosto de 2024**

In [ ]:
query_10 = '''SELECT *
                FROM consultas
                WHERE id_medico = 3
                AND EXTRACT(MONTH FROM DATE(fecha)) = 8
                AND EXTRACT(YEAR FROM DATE(fecha)) = 2024;'''
                
pd.read_sql(query_10, engine, index_col="id_consulta")

,id_paciente,id_medico,fecha,diagnostico,tratamiento,snomed_codigo
id_consulta,,,,,,
24,16,3,2024-08-08,Dolor de pecho,None,29857009
32,18,3,2024-08-16,Hipertensión arterial,None,59621000
45,5,3,2024-08-29,Dolor de pecho,None,29857009


**11. Obtener el nombre de los pacientes junto con la fecha y el diagnóstico de todas las consultas médicas realizadas en agosto del 2024.**

In [7]:
query_11 = '''SELECT c.id_consulta, p.nombre, c.fecha, c.diagnostico
                FROM consultas c JOIN pacientes p
                ON c.id_paciente = p.id_paciente
                WHERE EXTRACT(MONTH FROM DATE(fecha)) = 8
                AND EXTRACT(YEAR FROM DATE(fecha)) = 2024;'''
                
pd.read_sql(query_11, engine, index_col="id_consulta")

,nombre,fecha,diagnostico
id_consulta,,,
17,Esteban Muñoz,2024-08-01,Bronquitis
18,María Luisa Torres,2024-08-02,Luxación de hombro
19,Sofía Maldonado,2024-08-03,Sinusitis
20,Juan Pérez,2024-08-04,Fractura de pierna
21,Agustín Romero,2024-08-05,Ansiedad
22,Florencia Álvarez,2024-08-06,Luxación de hombro
23,Gabriela Vázquez,2024-08-07,Eccema
24,Gustavo Suárez,2024-08-08,Dolor de pecho
25,Clara Fernández,2024-08-09,Gastritis


**12. Obtener el nombre de los medicamentos prescritos más de una vez por el médico con ID igual a 2.**

In [9]:
query_12 = '''SELECT m.nombre, r.id_medico, COUNT(r.id_receta)
FROM medicamentos m JOIN recetas r 
ON r.id_medicamento = m.id_medicamento
WHERE r.id_medico = 2
GROUP BY m.nombre, r.id_medico
HAVING COUNT(r.id_receta) > 1;'''

pd.read_sql(query_12, engine)

,nombre,id_medico,count
0,Omeprazol,2,2


**13. Obtener el nombre de los pacientes junto con la cantidad total de recetas que han recibido.**

In [13]:
query_13 = ''' SELECT p.id_paciente, p.nombre, COUNT(r.id_receta) as cant_recetas
FROM pacientes p JOIN recetas r 
ON p.id_paciente = r.id_paciente
GROUP BY p.id_paciente, p.nombre;
'''
pd.read_sql(query_13, engine)

,id_paciente,nombre,cant_recetas
0,8,Esteban Muñoz,1
1,11,María Luisa Torres,2
2,19,Sofía Maldonado,2
3,4,Marcos Ramírez,2
4,21,Claudia Rojas,1
5,14,Nicolás Morales,2
6,3,Clara Fernández,1
7,17,Paula Medina,2
8,22,Juan Pérez,1
9,20,Facundo Paredes,1


**14.  Obtener el nombre del medicamento más recetado junto con la cantidad de recetas emitidas para ese medicamento  .**

In [7]:
query_14 = ''' SELECT m.nombre, COUNT(r.id_medicamento) AS total_recetas
FROM medicamentos m
JOIN recetas r ON m.id_medicamento = r.id_medicamento
GROUP BY m.nombre
ORDER BY total_recetas DESC
LIMIT 1;;
'''
pd.read_sql(query_14, engine)

,nombre,total_recetas
0,Férula,4


**15.  Obtener el nombre del paciente junto con la fecha de su última consulta y el diagnóstico asociado .** 

In [9]:
query_15= ''' SELECT p.nombre, MAX(c.fecha) AS ultima_consulta, c.diagnostico
FROM pacientes p
JOIN consultas c ON p.id_paciente = c.id_paciente
GROUP BY p.nombre, c.diagnostico;
'''
pd.read_sql(query_15, engine)

,nombre,ultima_consulta,diagnostico
0,Nicolás Morales,2024-07-28,Acné severo
1,Fernando García,2024-08-17,Trastorno obsesivo-compulsivo
2,Gabriela Vázquez,2024-08-07,Eccema
3,Gabriela Vázquez,2024-07-13,Glaucoma
4,Sofía Maldonado,2024-08-25,Ansiedad generalizada
...,...,...,...
56,Joaquín Castillo,2024-07-06,Acné
57,María Luisa Torres,2024-07-15,Depresión
58,Juan Pérez,2024-08-26,Dermatitis atópica
59,Agustín Romero,2024-08-16,Hipertensión arterial
